# NLP

In [1]:
corpus = ['the woman is a wise queen',
          'the man is a wise president',
          'she is a pretty woman',
          'he is a strong man',
          'she is still young',
          'he is very old',
          'he is the current president of US',
          'the prince is a son of the king',
          'the princess is a daughter of the king',
          'a prince is a young man',
          'a princess is a young woman']

In [2]:
# q1) 주석 부분에 function1의 목적을 쓰고, 그 주된 목적에 맞게 function1의 이름 변경하기
'''
function1의 목적 : target word와 neighbor word 매칭
'''
def w2v_input(corpus):    
    sentences = []
    for sentence in corpus:
        sentences.append(sentence.split())

    w_size = 2 # q2) s의 의미를 적고 그에 맞게 변수명 변경하기
    '''
    s의 의미 : window size
    '''
    data = []
    for sentence in sentences:
        for idx, word in enumerate(sentence):
            for neighbor in sentence[max(idx - w_size, 0) : min(idx + w_size, len(sentence)) + 1] : 
                if neighbor != word:
                    data.append([word, neighbor])
    return data

In [3]:
import pandas as pd

data = w2v_input(corpus)
df = pd.DataFrame(data, columns = ['input', 'label'])

In [4]:
df.head()

,input,label
0,the,woman
1,the,is
2,woman,the
3,woman,is
4,woman,a


In [5]:
df.shape

(194, 2)

In [6]:
# q3) 주석 부분에 function2의 목적을 쓰고, 그 목적에 맞게 function2의 이름 변경하기
'''
function2의 목적 : corpus에 등장하는 모든 단어의 집합을 만드는 함수 (중복 x)
'''
def word_set(corpus):
    words = []
    for text in corpus:
        for word in text.split(' '):
            words.append(word)
    words = set(words)
    return words

In [7]:
words = word_set(corpus)
words

{'US',
 'a',
 'current',
 'daughter',
 'he',
 'is',
 'king',
 'man',
 'of',
 'old',
 'president',
 'pretty',
 'prince',
 'princess',
 'queen',
 'she',
 'son',
 'still',
 'strong',
 'the',
 'very',
 'wise',
 'woman',
 'young'}

In [8]:
# q4) 주석 부분에 function3의 목적을 쓰고, 그 목적에 맞게 function3과 d의 이름을 변경하기
'''
function3의 목적 : 각 단어들을 key로 나열순서 index를 value에 저장하는 dict를 만드는 함수
'''
def word_indexing(words):
    word_index = {}
    for i,word in enumerate(words):
        word_index[word] = i
    return word_index

In [10]:
d = word_indexing(words)
d

{'man': 0,
 'old': 1,
 'he': 2,
 'still': 3,
 'current': 4,
 'is': 5,
 'the': 6,
 'princess': 7,
 'queen': 8,
 'strong': 9,
 'very': 10,
 'woman': 11,
 'of': 12,
 'king': 13,
 'son': 14,
 'wise': 15,
 'daughter': 16,
 'a': 17,
 'pretty': 18,
 'young': 19,
 'US': 20,
 'prince': 21,
 'president': 22,
 'she': 23}

In [11]:
import numpy as np

# q5) 주석 부분에 function4의 목적을 쓰고, 그 목적에 맞게 function4와 ohe의 이름 변경하기
'''
function4의 목적 : one hot encoding 함수
'''
def one_hot_encoding(word_index, ONE_HOT_DIM):
    ohe = np.zeros(ONE_HOT_DIM)
    ohe[word_index] = 1
    return ohe

In [12]:
# q6) Word2Vec을 tensorflow로 구현한 코드에서 ? 부분을 올바르게 채워넣기
# q7) 여기서 구현한 Word2Vec의 아키텍쳐는 CBOW or Skip Gram ?
'''
Skip Gram
'''
# cross entropy 참고
# https://ratsgo.github.io/deep%20learning/2017/10/02/softmax/
# https://kevinthegrey.tistory.com/123

'\nSkip Gram\n'

In [22]:
import tensorflow as tf

ONE_HOT_DIM = len(words)

X = [] # input word
Y = [] # target word

for x, y in zip(df['input'], df['label']):
    X.append(one_hot_encoding(d[x], ONE_HOT_DIM))
    Y.append(one_hot_encoding(d[y], ONE_HOT_DIM))
    
    # convert X,Y to numpy arrays
X_train = np.asarray(X)
Y_train = np.asarray(Y)


In [27]:
len(words)

24

In [31]:
import tensorflow as tf

ONE_HOT_DIM = len(words)

X = [] # input word
Y = [] # target word

for x, y in zip(df['input'], df['label']):
    X.append(one_hot_encoding(d[x], ONE_HOT_DIM))
    Y.append(one_hot_encoding(d[y], ONE_HOT_DIM))

# convert X,Y to numpy arrays
X_train = np.asarray(X)
Y_train = np.asarray(Y)

# placeholders for X_train and Y_train
x = tf.placeholder(tf.float32, shape=(None, ONE_HOT_DIM))
y_label = tf.placeholder(tf.float32, shape=(None, ONE_HOT_DIM))

# embedding dimension
EMBEDDING_DIM = 2

# hidden layer : represent word vector eventually
W1 = tf.Variable(tf.random_normal([ONE_HOT_DIM, EMBEDDING_DIM]))
b1 = tf.Variable(tf.random_normal([1]))
hidden_layer = tf.add(tf.matmul(x,W1), b1)

# output layer
W2 = tf.Variable(tf.random_normal([EMBEDDING_DIM, ONE_HOT_DIM]))
b2 = tf.Variable(tf.random_normal([1]))
output = tf.nn.softmax(tf.add( tf.matmul(hidden_layer, W2), b2))

# loss function : cross entropy
loss = tf.reduce_mean(-tf.reduce_sum(y_label * tf.log(output), axis=[1]))

# training
train = tf.train.GradientDescentOptimizer(0.05).minimize(loss)

In [32]:
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init) 

iteration = 10000
for i in range(iteration):
    # input : X_train which is one hot encoded word
    # label : Y_train which is one hot encoded neighbor word
    sess.run(train, feed_dict={x: X_train, y_label: Y_train})
    if i % 2000 == 0:
        print('iteration '+ str(i) +' loss is : ', sess.run(loss, feed_dict={x: X_train, y_label: Y_train}))

iteration 0 loss is :  6.7417226
iteration 2000 loss is :  2.5378864
iteration 4000 loss is :  2.47047
iteration 6000 loss is :  2.442094
iteration 8000 loss is :  2.422756


In [33]:
# the hidden layer (W1 + b1) -> look up table
vectors = sess.run(W1 + b1)
vectors

array([[-1.9886237 ,  0.5578252 ],
       [-1.0275269 , -0.59022737],
       [-1.6285518 , -1.4305437 ],
       [-1.2066599 ,  0.905879  ],
       [-0.98765403, -3.1654143 ],
       [-0.8895926 ,  0.15903425],
       [-0.48015952, -1.4624747 ],
       [-2.7858648 , -1.3706934 ],
       [-1.2663746 , -0.38706785],
       [-1.5755341 , -1.0651104 ],
       [-0.5117104 ,  0.18440795],
       [-1.9315703 ,  0.46140575],
       [-0.19389528, -1.0478638 ],
       [-1.031911  , -3.2740448 ],
       [-1.2380123 , -2.4367032 ],
       [-0.94369256, -1.6140618 ],
       [-1.3060849 , -2.6613817 ],
       [-0.56807786, -0.5422425 ],
       [-1.5044446 , -0.84225255],
       [-1.3898277 , -0.90263087],
       [ 0.50633895, -2.7538352 ],
       [-2.8421426 , -1.4314857 ],
       [-0.31798965, -1.018264  ],
       [-1.6272169 , -1.0346696 ]], dtype=float32)

In [34]:
sess.close()

In [35]:
w2v_df = pd.DataFrame(vectors, columns = ['x1', 'x2'])
w2v_df['word'] = words
w2v_df = w2v_df[['word', 'x1', 'x2']]
w2v_df

,word,x1,x2
0,man,-1.988624,0.557825
1,old,-1.027527,-0.590227
2,he,-1.628552,-1.430544
3,still,-1.206660,0.905879
4,current,-0.987654,-3.165414
5,is,-0.889593,0.159034
6,the,-0.480160,-1.462475
7,princess,-2.785865,-1.370693
8,queen,-1.266375,-0.387068
9,strong,-1.575534,-1.065110


In [36]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots()

for word, x1, x2 in zip(w2v_df['word'], w2v_df['x1'], w2v_df['x2']):
    ax.annotate(word, (x1,x2 ))
    
PADDING = 1.0
x1_axis_min = np.amin(vectors, axis=0)[0] - PADDING
x2_axis_min = np.amin(vectors, axis=0)[1] - PADDING
x1_axis_max = np.amax(vectors, axis=0)[0] + PADDING
x2_axis_max = np.amax(vectors, axis=0)[1] + PADDING
 
plt.xlim(x1_axis_min,x1_axis_max)
plt.ylim(x2_axis_min,x2_axis_max)
plt.rcParams["figure.figsize"] = (10,10)

plt.show()

<Figure size 640x480 with 1 Axes>